# Extract candidate gene info from *Chlamy* gff

gff file path: `scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/Creinhardtii_v5.3_223_gene.gff3.gz `

Target data structure:


In [290]:
import gzip
import pandas as pd
import re
import time

In [292]:
merged = pd.read_csv("../../data/intermediate_data_from_gff/merged_v5_3_1.csv")
gff_file_path='/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/Creinhardtii_v5.3_223_gene.gff3.gz'


In [293]:
def BinarySearch(lys, val):
    
    '''requires re'''
    
    '''This function returns the position of the element in the list lys that contains the string pattern lys. If no match 
    
    Usage: lys = list of strings to search through; val = string pattern to search for
    
    Warning: This function only works when the beginning of the string matches val'''
        
    first = 0
    last = len(lys)-1
    index = -1
    
    
    p = re.compile(re.escape(val))
    
    
    while (first <= last) and (index == -1):
        mid = round((first+last)/2)
        
        index_list = sorted([lys[mid],val])
        
        if p.match(lys[mid]):
            index = mid
        else:
            
            if index_list.index(val)<index_list.index(lys[mid]):
                last = mid -1
                
            else:
                first = mid +1
                
    return index

In [294]:
def tryextract(i, pattern):
    
    try: 
        m = re.search(pattern, i).group(1)
        return(m)
    
    except AttributeError:
        return(None)

In [295]:

with gzip.open(gff_file_path, "rt", encoding="utf-8") as z:
    
    df = pd.read_csv(z,delimiter=r"\s+",skiprows=1,header=None)
    df.columns = ['chromosome', 'source', 'feature', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    df['Name'] = df.attributes.apply(lambda x: tryextract(x, r"Name=(.+);pacid"))
    #df['pacid'] = df.attributes.apply(lambda x: tryextract(x, r"pacid=(.+);longest"))
    df['ID'] = df.attributes.apply(lambda x: tryextract(x, r"ID=(PAC:[0-9]+)"))
        #except AttributeError: 
    #del df['attributes']
    #df = df.sort_values(by="Name")
    

In [297]:
subset = df[df.feature =="mRNA"].sort_values(by="Name").reset_index()
search_list = list(subset.Name)

In [298]:
t0 = time.time()
pacid_list = []

for row in range(0,len(merged)):
    transcript_id = merged.loc[row,'transcript_id_v5.3.1']
    pacid = None
    
    index = BinarySearch(search_list, transcript_id)
    
    if index > -1: 
        pacid = subset.loc[index, "ID"]
                    
    pacid_list.append(pacid)

t1 = time.time()
print("Time taken", t1-t0, "s")

Time taken 10.233540534973145 s


In [299]:
merged['PAC_ID'] = pacid_list

In [300]:
groupby_obj = df[df.feature =="CDS"].groupby("ID")
gff_ext_dict = {}
#search_list = list(subset.ID)

In [301]:
for name, group in groupby_obj:
    
    
    gff_ext_dict[name] = group
    

In [307]:
print(list(merged.PAC_ID).count(None),"transcripts out of", len(merged), "transcripts did not have a matching PAC ID")

176 transcripts out of 17956 transcripts did not have a matching PAC ID


In [303]:
df.attributes[:10]

0                  ID=Cre01.g000050;Name=Cre01.g000050
1    ID=PAC:26903339;Name=Cre01.g000050.t1.3;pacid=...
2    ID=PAC:26903339.five_prime_UTR.1;Parent=PAC:26...
3    ID=PAC:26903339.CDS.1;Parent=PAC:26903339;paci...
4    ID=PAC:26903339.CDS.2;Parent=PAC:26903339;paci...
5    ID=PAC:26903339.CDS.3;Parent=PAC:26903339;paci...
6    ID=PAC:26903339.CDS.4;Parent=PAC:26903339;paci...
7    ID=PAC:26903339.three_prime_UTR.1;Parent=PAC:2...
8                  ID=Cre01.g000100;Name=Cre01.g000100
9    ID=PAC:26903974;Name=Cre01.g000100.t1.3;pacid=...
Name: attributes, dtype: object